<a href="https://colab.research.google.com/gist/klarteno/0eee61b980f8feca34c731338f03b5b7/copy-of-aml-final-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### use the .py version of this for import in notebooks


In [1]:
import torch
import numpy as np
import pandas as pd
import sys
import os

from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder #, OneHotEncoder

from pprint import pprint
import time
import import_ipynb


c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive/My Drive/1_MalmoUni/AdvML/Project'
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n%cd '/content/drive/My Drive/1_MalmoUni/AdvML/Project'\n"

In [3]:
if torch.cuda.is_available():
   torch.cuda.empty_cache() 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('DEVICE', DEVICE) 

import random, torch, os, numpy as np

def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

DEVICE cuda


## Store All CSV Filenames in the Dataset in an Array

In [6]:
# root_dir = 'CICIDS2017'
csv_files = []
for dirname, _, filenames in os.walk('C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE'):
  # yelds a 3-tuple (dirpath, (sub)dirnmes, filenames)
    for filename in filenames:
        csv_file = os.path.join(dirname, filename)
        print(csv_file)
        csv_files.append(csv_file)

C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv
C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv
C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
C:/Users/androgo/Documents/Python Scripts/IDS/CICIDS2017_notebooks/IDS_Project/datasets/MachineLearningCSV/MachineLearningCVE\Thursd

## Concatenate the data from all CSV files into one Pandas DataFrame

In [7]:
week_data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
#print("original length of week_data:", len(week_data))
print(week_data.shape)

(2830743, 79)


In [ ]:
def print_nans():
    print(type(week_data))
    result = week_data.isna().sum()
    for idx in result.index:
        if result[idx] > 0:
            print(idx,':' ,result[idx])
            
print_nans()            

<class 'pandas.core.frame.DataFrame'>
Flow Bytes/s : 1358


## Data Preprocessing
Dropping lines that contain invalid values

In [ ]:
week_data.columns = week_data.columns.str.strip()
week_data.replace([np.inf, -np.inf],  np.nan, inplace=True)  # replace -infinity and+infinity with NaN
week_data.dropna(inplace=True)    #remove missing values
# print("Length of week_data after droping null values:", len(week_data))

<class 'pandas.core.frame.DataFrame'>
Flow Bytes/s : 2867
Flow Packets/s : 2867


### Print Info - Classes in the Dataset

###### - when assessing the distribution of labels, it turns out that the classes are inbalanced and this id expected for network data

In [ ]:
print("\n Number of Labels: ",week_data.Label.nunique())
print("Nuber or records/observations per label:\n", week_data.Label.value_counts())
#week_data.head()

BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

 Number of Labels:  15


In [ ]:
# iterating the columns
for col in week_data.columns:
    print(col)

Destination Port
Flow Duration
Total Fwd Packets
Total Backward Packets
Total Length of Fwd Packets
Total Length of Bwd Packets
Fwd Packet Length Max
Fwd Packet Length Min
Fwd Packet Length Mean
Fwd Packet Length Std
Bwd Packet Length Max
Bwd Packet Length Min
Bwd Packet Length Mean
Bwd Packet Length Std
Flow Bytes/s
Flow Packets/s
Flow IAT Mean
Flow IAT Std
Flow IAT Max
Flow IAT Min
Fwd IAT Total
Fwd IAT Mean
Fwd IAT Std
Fwd IAT Max
Fwd IAT Min
Bwd IAT Total
Bwd IAT Mean
Bwd IAT Std
Bwd IAT Max
Bwd IAT Min
Fwd PSH Flags
Bwd PSH Flags
Fwd URG Flags
Bwd URG Flags
Fwd Header Length
Bwd Header Length
Fwd Packets/s
Bwd Packets/s
Min Packet Length
Max Packet Length
Packet Length Mean
Packet Length Std
Packet Length Variance
FIN Flag Count
SYN Flag Count
RST Flag Count
PSH Flag Count
ACK Flag Count
URG Flag Count
CWE Flag Count
ECE Flag Count
Down/Up Ratio
Average Packet Size
Avg Fwd Segment Size
Avg Bwd Segment Size
Fwd Header Length.1
Fwd Avg Bytes/Bulk
Fwd Avg Packets/Bulk
Fwd Avg Bulk Ra

Check if there are features that are not numbers

In [ ]:
string_features = list(week_data.select_dtypes(include=['object']).columns)
string_features.remove('Label')
string_features  

[]

##### Check if somecolumns have the right data type

In [ ]:
# week_data2 = week_data.drop(week_data[pd.isnull(week_data['Flow ID'])].index)
print(week_data["Flow Bytes/s"].dtype)
print(week_data["Flow Packets/s"].dtype)

float64
float64


## Inputs-Output Data Split

In [ ]:
y = week_data.Label
X = week_data.drop(columns='Label')

class_labels = y.unique()

num_classes = y.nunique()     # number of unique values
print("shape of X: ",X.shape)
print("number of labels of y: ", num_classes)
print("Class labels: ", class_labels)
print()

(2827876, 78)
number of labels: 15
Labels: ['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']
typeof labels: <class 'numpy.ndarray'>

Nuber or records/observations per label:
 BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64


In [ ]:
#  are there missing values
'''

from sklearn.impute import SimpleImputer

mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
cat_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
for c in X.columns:
  X[c] = mean_imp.fit_transform(X[[c]]).ravel()
  
'''


"\n\nfrom sklearn.impute import SimpleImputer\n\nmean_imp = SimpleImputer(missing_values=-1, strategy='mean')\ncat_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')\nfor c in X.columns:\n  X[c] = mean_imp.fit_transform(X[[c]]).ravel()\n  \n"

##### Select the number of relevant features out of total of 78

In [ ]:

selected_features = ["Bwd Packet Length Min", "Subflow Fwd Bytes", "Total Length of Fwd Packets", "Fwd Packet Length Mean", "Bwd Packet Length Std", "Flow IAT Min", "Fwd IAT Min", "Flow IAT Mean", "Flow Duration", "Flow IAT Std", "Init_Win_bytes_forward", "Active Min", "Active Mean", "Bwd Packets/s", "Bwd IAT Mean", "Fwd IAT Mean", "ACK Flag Count", "Fwd PSH Flags", "SYN Flag Count", "Fwd Packets/s", "Init_Win_bytes_backward", "PSH Flag Count", "Average Packet Size"]
X_select = X[np.intersect1d(X.columns, selected_features)]
X_select.head(2)

,ACK Flag Count,Active Mean,Active Min,Average Packet Size,Bwd IAT Mean,Bwd Packet Length Min,Bwd Packet Length Std,Bwd Packets/s,Flow Duration,Flow IAT Mean,...,Fwd IAT Min,Fwd PSH Flags,Fwd Packet Length Mean,Fwd Packets/s,Init_Win_bytes_backward,Init_Win_bytes_forward,PSH Flag Count,SYN Flag Count,Subflow Fwd Bytes,Total Length of Fwd Packets
0,1,0.0,0,9.0,0.0,0,0.0,0.000000,3,3.0,...,3,0,6.0,666666.666700,-1,33,0,0,12,12
1,1,0.0,0,9.0,0.0,6,0.0,9174.311927,109,109.0,...,0,0,6.0,9174.311927,256,29,0,0,6,6


In [ ]:
# sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
# *arrays - Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.
# test_size: float, between 0.0 and 1.0 Default: (1-train_size); if train_size is also None, test_size will be set to 0.25.
# train_size: default (1-test_size); 0.75
# random_state: Controls the shuffling applied to the data before applying the split ?Pass an int for reproducible output across multiple function calls? (random generator seed = 42 -(The Hitchhiker's Guide to the Galaxy))
# Returns: List containing train-test split of inputs. splitting: list, length=2 * len(arrays)

X_train, X_test, y_train, y_test = train_test_split(X_select, y, random_state=99, stratify=y)

print("\nafter spliting the data:")
print("training data shape:", X_train.shape)
print("test data shape:", X_test.shape)
print("training data shape:", y_train.shape)
print("test data shape:", y_test.shape)
X_train.head(2)


after spliting the data:
training data shape: (2120907, 23)
test data shape: (706969, 23)
training data shape: (2120907,)
test data shape: (706969,)


,ACK Flag Count,Active Mean,Active Min,Average Packet Size,Bwd IAT Mean,Bwd Packet Length Min,Bwd Packet Length Std,Bwd Packets/s,Flow Duration,Flow IAT Mean,...,Fwd IAT Min,Fwd PSH Flags,Fwd Packet Length Mean,Fwd Packets/s,Init_Win_bytes_backward,Init_Win_bytes_forward,PSH Flag Count,SYN Flag Count,Subflow Fwd Bytes,Total Length of Fwd Packets
1828626,0,62399.27273,42034,185.976744,5827397.368,0,548.659982,0.173027,115588963,2.752118e+06,...,4,0,57.913043,0.198981,1042,29200,1,0,1332,1332
2095131,0,0.00000,0,67.750000,48.000,83,0.000000,16528.925620,121,4.033333e+01,...,1,0,35.000000,16528.925620,-1,-1,0,0,70,70


##### Encode the text of the labels to numbers

In [ ]:
le = LabelEncoder()       # Encode target labels with value between 0 and n_classes-1

y_train_binary = le.fit_transform(y_train)

#print("instances per label in test set\n", y_test_binary.value_counts())
# transform -	Transform labels to normalized encoding.
y_test_binary = le.transform(y_test)

#we use fit_transform() on training data but transform() on the test data

# classes_ - ndarray of shape (n_classes,) - Holds the label for each class.
# To create a dictionary from two sequences, use dict(zip(keys, values))
# The zip(fields, values) method returns an iterator that generates two-items tuples 
labels_dict = dict(zip(le.classes_, range(len(le.classes_))))


pprint(labels_dict)

{'BENIGN': 0,
 'Bot': 1,
 'DDoS': 2,
 'DoS GoldenEye': 3,
 'DoS Hulk': 4,
 'DoS Slowhttptest': 5,
 'DoS slowloris': 6,
 'FTP-Patator': 7,
 'Heartbleed': 8,
 'Infiltration': 9,
 'PortScan': 10,
 'SSH-Patator': 11,
 'Web Attack � Brute Force': 12,
 'Web Attack � Sql Injection': 13,
 'Web Attack � XSS': 14}


##### Sample only trainning data an leave the test or validation data as is  
##### run it only if saved sampled data is missing

In [ ]:
from imblearn.under_sampling import InstanceHardnessThreshold

'''  
iht = InstanceHardnessThreshold(random_state=42, sampling_strategy='majority', n_jobs=3)
X_train_sampled, y_train_binary_sampled = iht.fit_resample(X_train, y_train_binary)
# os.makedirs('folder/subfolder', exist_ok=True)  # this is an example if folder is missing

# save what have been sampled to files and do not run it again
X_train_sampled.to_csv('datasets/X_train_sampled_23_features.csv')
np.savetxt("datasets/y_train_binary_sampled_23_features.csv", y_train_binary_sampled, delimiter=",")
'''

In [ ]:
type(X_train)
X_train.shape

(2120907, 23)

In [ ]:
X_train_sampled = pd.read_csv('datasets/X_train_sampled_23_features.csv')
y_train_binary_sampled = np.loadtxt('datasets/y_train_binary_sampled_23_features.csv', delimiter=',')

In [ ]:
#print(type(X_train_sampled))
print(X_train_sampled.shape)

# print(type(y_train_binary_sampled))
pprint(pd.DataFrame(y_train_binary_sampled).value_counts())

(2090544, 24)
0.0     1673127
4.0      172593
10.0     119103
2.0       96019
3.0        7720
7.0        5951
11.0       4423
6.0        4347
5.0        4124
1.0        1467
12.0       1130
14.0        489
9.0          27
13.0         16
8.0           8
dtype: int64


##### Check the files loaded because extra columns gets added

In [ ]:
# if  len(X_train_sampled.columns)==79:  
if  len(X_train_sampled.columns) == 24:
    X_train_sampled['Unnamed: 0']  
    X_train_sampled = X_train_sampled.drop(columns='Unnamed: 0')

In [ ]:

# it run for 483 minutes then I interapted
'''
from imblearn.over_sampling import SVMSMOTE 

sm = SVMSMOTE(random_state=42, n_jobs=2)

# X_sampled, y_sampled = sm.fit_resample(X.to_numpy(), y_binary.to_numpy())
X_train, y_sampled = sm.fit_resample(X_train, y_train_binary) 
'''
# to try
'''
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
X, y = smt.fit_sample(X, y)
'''

'\nfrom imblearn.over_sampling import SVMSMOTE \n\nsm = SVMSMOTE(random_state=42, n_jobs=2)\n\n# X_sampled, y_sampled = sm.fit_resample(X.to_numpy(), y_binary.to_numpy())\nX_train, y_sampled = sm.fit_resample(X_train, y_train_binary) \n'

In [ ]:
scaler = MinMaxScaler()

# scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_sampled)
X_test = scaler.transform(X_test)

In [ ]:
class SecurityDataset(torch.utils.data.Dataset):

  def __init__(self, X_train, y_train_binary):
    
    self.X_train = torch.tensor(X_train, dtype = torch.float32)
    self.Y_train = torch.tensor(y_train_binary)  

  def __len__(self):
    return len(self.Y_train)
  
  def __getitem__(self, index):
    return self.X_train[index], self.Y_train[index]
  

In [ ]:
train_dataset = SecurityDataset(torch.tensor(X_train_sampled.values), y_train_binary_sampled)
test_dataset = SecurityDataset(X_test, y_test_binary)

## test :
dataiter = iter(train_dataset)
x_data, y_data = dataiter.next()

print('Type of x_data: ', type(x_data))
print('shape of x_data: ', x_data.shape)

print('Type of y_data: ', type(y_data))
print('Shape of y_data: ', y_data.shape)

C:\Users\androgo\AppData\Local\Temp\ipykernel_27100\2872934758.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X_train = torch.tensor(X_train, dtype = torch.float32)


### Class Weights

In [ ]:
from sklearn.utils import class_weight

classes_y = np.array(list(labels_dict.values()))
print('classes_y: ',classes_y)

#calculate the class weights
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = classes_y, # np.unique(y_train_binary),
                                                 y = y_train_binary_sampled)
print('class_weights: ',class_weights)
print()
classes_class_weights = dict(zip(classes_y, class_weights))
print('classes_class_weights: ',classes_class_weights)

classes_y:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
class_weights:  [8.32988769e-02 9.50031357e+01 1.45147939e+00 1.80530570e+01
 8.07504360e-01 3.37947624e+01 3.20610996e+01 2.34195261e+01
 1.74212000e+04 5.16183704e+03 1.17016028e+00 3.15101967e+01
 1.23335929e+02 8.71060000e+03 2.85009407e+02]

classes_class_weights:  {0: 0.08329887689338586, 1: 95.00313565098841, 2: 1.4514793947031317, 3: 18.053056994818654, 4: 0.807504359968249, 5: 33.79476236663434, 6: 32.061099608925694, 7: 23.419526130062174, 8: 17421.2, 9: 5161.837037037037, 10: 1.1701602814370755, 11: 31.510196699073028, 12: 123.33592920353982, 13: 8710.6, 14: 285.0094069529652}


In [ ]:
# class_weights.round(decimals=3, out=None)
class_weights = np.around(class_weights, decimals=3)

##### Apply WeightedRandomSampler only to train data and leave test or validate data untouched because is treated as unseen

In [ ]:
weights_sampler = 1. / class_weights
sample_weights = [0] * len(train_dataset)
# weights_sampler =np.around(weights_sampler, decimals=5)

for idx, (data, label) in enumerate(train_dataset):
        class_weight = class_weights[ int(label.item()) ]
        sample_weights[idx] = class_weight
        
sampler = torch.utils.data.WeightedRandomSampler(sample_weights, num_samples=
                                    len(sample_weights), replacement=True)

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=sampler, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# torch.save(train_loader, 'out_mlp/train_loader.pth')
# train_loader2 = torch.load('out_mlp/train_loader.pth') 

torch.utils.data.dataloader.DataLoader

##### Do not run because it takes a lot of time , run only if the output of the cell is not displayed

In [ ]:
''' 
labels_count = [0] * num_classes

for i in range(15):
        labels_count[i]=i

for epoch in range(10):
    for data, labels in train_loader:
        for i in range(num_classes):
                labels_count[i] += torch.sum(labels==i)
        
print(labels_count)
print()
print()
print(max(labels_count))
print(min(labels_count))
'''

[tensor(1388070), tensor(1395141), tensor(1394069), tensor(1394077), tensor(1395305), tensor(1395214), tensor(1392920), tensor(1394196), tensor(1390981), tensor(1394197), tensor(1394975), tensor(1395769), tensor(1393125), tensor(1394618), tensor(1392728)]


tensor(1395769)
tensor(1388070)


##### Run only for test if nan are present in data 

In [ ]:
import tqdm

for step, (batch_x, batch_y) in enumerate(tqdm.tqdm(train_loader),0):
            # zero the parameter gradients
            batch_x, batch_y = batch_x.to(DEVICE), batch_y.to(DEVICE)
            
            # batch_x = batch_x.view(-1, input_size)
            # batch_x = batch_x.float()
            assert not torch.isnan(batch_x).any()
            assert  torch.isfinite(batch_x).all()
            
            assert not torch.isnan(batch_y).any()
            assert torch.isfinite(batch_y).all()

100%|██████████| 65329/65329 [02:26<00:00, 444.72it/s]


In [ ]:
def get_dataloaders():
    
    return train_loader,test_loader

In [ ]:
def get_dataloaders2():
    
    return 'train_loader,test_loader'